[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klar74/WS2025_lecture/blob/main/Vorlesung_28/VL28_Word_Embeddings.ipynb)

# VL28 Notebook 1: Word Embeddings
## Von One-Hot zu Word2Vec

In diesem Notebook lernen wir:
- Wie One-Hot Encoding funktioniert (und warum es nicht ausreicht)
- Wie Word2Vec Bedeutung durch Position im Vektorraum kodiert
- Das berühmte Beispiel: König - Mann + Frau ≈ Königin
- Die Limitation: Ein Wort = Ein Vektor (kein Kontext)

In [ ]:
# Installation (nur einmal ausführen)
# !pip install gensim numpy matplotlib scikit-learn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings('ignore')

## 1. One-Hot Encoding: Die naive Lösung

Jedes Wort bekommt eine Position in einem Vektor. Nur diese Position ist 1, alle anderen 0.

In [ ]:
# Beispiel-Vokabular
vocab = ["Hund", "Katze", "Vogel", "Auto"]

# One-Hot Vektoren erstellen
def one_hot(word, vocab):
    vec = np.zeros(len(vocab))
    vec[vocab.index(word)] = 1
    return vec

# Vektoren ausgeben
for word in vocab:
    print(f"{word:10s}: {one_hot(word, vocab)}")

**Problem:** Alle Wörter haben gleichen Abstand zueinander!
- Abstand Hund-Katze = Abstand Hund-Auto
- Keine Bedeutung kodiert

## 2. Word2Vec: Eigenes Training auf Beispiel-Korpus

**Realität:** Word2Vec braucht normalerweise VIELE Daten (Millionen Sätze aus Wikipedia, Nachrichten, etc.).

**Unser Ansatz:** Wir trainieren ein **eigenes kleines Modell** auf einem deutschen Beispiel-Korpus!

⚠️ **Wichtig:** Mit unserem Mini-Korpus lernt das Modell nur die Grundprinzipien. Für echte Anwendungen würde man vortrainierte Modelle nutzen oder auf sehr großen Textsammlungen trainieren.

In [ ]:
# Eigenen deutschen Trainingskorpus erstellen
# In der Praxis würde man auf Wikipedia, Nachrichten, Büchern trainieren

# Deutscher Trainingskorpus mit vielen Beispielsätzen
base_sentences = [
    # Tiere
    ["der", "hund", "bellt", "laut"],
    ["die", "katze", "miaut", "leise"],
    ["der", "hund", "jagt", "die", "katze"],
    ["der", "vogel", "singt", "schön"],
    ["der", "hund", "ist", "ein", "tier"],
    ["die", "katze", "ist", "ein", "tier"],
    
    # Fahrzeuge
    ["das", "auto", "fährt", "schnell"],
    ["das", "fahrrad", "ist", "langsam"],
    
    # König/Königin - Mann/Frau (VIELE Beispiele!)
    ["der", "könig", "regiert", "das", "land"],
    ["die", "königin", "regiert", "das", "land"],
    ["könig", "und", "königin", "sind", "verheiratet"],
    ["der", "könig", "ist", "ein", "mann"],
    ["die", "königin", "ist", "eine", "frau"],
    ["ein", "mann", "arbeitet", "hart"],
    ["eine", "frau", "arbeitet", "hart"],
    ["der", "mann", "geht", "spazieren"],
    ["die", "frau", "geht", "spazieren"],
    ["jeder", "mann", "hat", "rechte"],
    ["jede", "frau", "hat", "rechte"],
    
    # Weitere Paare
    ["der", "vater", "ist", "ein", "mann"],
    ["die", "mutter", "ist", "eine", "frau"],
    ["vater", "und", "mutter", "sind", "eltern"],
    ["der", "bruder", "ist", "männlich"],
    ["die", "schwester", "ist", "weiblich"],
    ["mein", "bruder", "hilft", "mir"],
    ["meine", "schwester", "hilft", "mir"],
    ["der", "onkel", "ist", "verwandt"],
    ["die", "tante", "ist", "verwandt"],
    
    # Adjektive für König/Königin
    ["der", "mächtige", "könig", "herrscht"],
    ["die", "mächtige", "königin", "herrscht"],
    ["ein", "weiser", "könig", "entscheidet"],
    ["eine", "weise", "königin", "entscheidet"],
]

# Erweitere Korpus durch Variationen (mehr Kontext)
sentences = base_sentences * 20  # Wiederholen für mehr Training

print(f"Trainingskorpus erstellt: {len(sentences)} Sätze")
print("Starte Training...")

# Word2Vec trainieren mit optimierten Parametern
model = Word2Vec(
    sentences, 
    vector_size=150,
    window=7,
    min_count=1, 
    sg=1,                 # Skip-Gram (besser für kleine Korpora)
    epochs=500,
    negative=10,          # Negative Sampling
    alpha=0.025,          # Learning rate
    min_alpha=0.0001      # Minimum learning rate
)

print("✓ Modell trainiert!")
print(f"  Vokabular-Größe: {len(model.wv)} Wörter")
print(f"  Trainings-Sätze: {len(sentences)}")
print(f"  Dimensionen: {model.vector_size}")

## 3. Vektor-Arithmetik: Das magische Beispiel

**König - Mann + Frau ≈ Königin**

Word2Vec kann Bedeutungsrelationen als Vektoren lernen - **aber nur mit genug Trainingsdaten!**

⚠️ **Wichtig:** Mit unserem Mini-Korpus funktioniert das nur begrenzt. In der Praxis trainiert man auf Millionen von Sätzen (z.B. Wikipedia).

### Was ist `model.wv`?

**`model.wv`** = **Word Vectors** (KeyedVectors-Objekt)

Nach dem Training sind alle gelernten Embeddings hier gespeichert:
- Wie ein Wörterbuch: `Wort → Vektor` (numpy array mit 150 Dimensionen)
- Zugriff auf einzelne Vektoren: `model.wv['könig']` gibt einen Vektor mit 150 Zahlen

### Was ist `model.wv.most_similar()`?

Findet die **ähnlichsten Wörter** basierend auf **Cosinus-Ähnlichkeit** der Vektoren.

**Zwei Modi:**

**1️⃣ Einfach - Ähnliche Wörter finden:**
```python
model.wv.most_similar('könig', topn=5)
# → [('königin', 0.95), ('herrscht', 0.82), ...]
```

**2️⃣ Vektor-Arithmetik - Das ist die Magie!**
```python
model.wv.most_similar(
    positive=['könig', 'frau'],  # Addiere diese Vektoren
    negative=['mann'],           # Subtrahiere diesen Vektor
    topn=5                       # Top 5 Ergebnisse
)
```

**Intern passiert:**
1. Berechne: `vektor_könig + vektor_frau - vektor_mann`
2. Finde Wörter, deren Vektoren diesem Ergebnis am ähnlichsten sind
3. Gib Liste `[(wort, ähnlichkeit), ...]` zurück

**Warum funktioniert das?**
- `vektor_könig - vektor_mann` ≈ "Royalität" (ohne Geschlecht)
- `+ vektor_frau` = "Royalität" + "weiblich" = **Königin**!

Word2Vec lernt **semantische Richtungen** im Vektorraum!

In [ ]:
# Vektor-Arithmetik testen
print("=" * 60)
print("VEKTOR-ARITHMETIK: König - Mann + Frau ≈ Königin")
print("=" * 60)

# Test 1: König - Mann + Frau
try:
    result = model.wv.most_similar(
        positive=['könig', 'frau'], 
        negative=['mann'],
        topn=5
    )
    print("\n✓ König - Mann + Frau ≈")
    for word, score in result:
        if word not in ['könig', 'frau', 'mann']:  # Zeige nicht die Eingabewörter
            print(f"  {word:15s} (Ähnlichkeit: {score:.3f})")
except Exception as e:
    print(f"\n✗ Fehler: {e}")

print("\n" + "=" * 60)

# Test 2: Andere Geschlechter-Paare
print("\nWeitere Relationen:")
try:
    # Vater - Mann + Frau ≈ Mutter
    result = model.wv.most_similar(
        positive=['vater', 'frau'], 
        negative=['mann'],
        topn=3
    )
    print("\n  Vater - Mann + Frau ≈")
    for word, score in result:
        if word not in ['vater', 'frau', 'mann']:
            print(f"    {word:15s} ({score:.3f})")
except:
    pass

try:
    # Bruder - Mann + Frau ≈ Schwester
    result = model.wv.most_similar(
        positive=['bruder', 'frau'], 
        negative=['mann'],
        topn=3
    )
    print("\n  Bruder - Mann + Frau ≈")
    for word, score in result:
        if word not in ['bruder', 'frau', 'mann']:
            print(f"    {word:15s} ({score:.3f})")
except:
    pass

print("\n" + "=" * 60)
print("\n💡 REALITÄT: Für stabile Relationen braucht Word2Vec VIEL mehr Daten!")
print("   Echte Modelle trainieren auf Wikipedia (Millionen Sätze).")
print("   Unser Mini-Korpus zeigt nur das PRINZIP.")

## 4. Visualisierung: Wörter im 2D-Raum

Wir reduzieren die Dimensionen auf 2D mit t-SNE und plotten die Wörter.

In [ ]:
# Alle Wörter und ihre Vektoren extrahieren
words = list(model.wv.index_to_key)
vectors = np.array([model.wv[word] for word in words])

# t-SNE: Dimensionsreduktion auf 2D
tsne = TSNE(n_components=2, random_state=42, perplexity=min(10, len(words)-1))
vectors_2d = tsne.fit_transform(vectors)

# Plot erstellen
plt.figure(figsize=(14, 10))
plt.scatter(vectors_2d[:, 0], vectors_2d[:, 1], alpha=0.5, s=100)

# Wichtige Wörter hervorheben
wichtige_worte = ['könig', 'königin', 'mann', 'frau', 'vater', 'mutter', 'bruder', 'schwester']
for i, word in enumerate(words):
    if word in wichtige_worte:
        plt.annotate(word, (vectors_2d[i, 0], vectors_2d[i, 1]), 
                    fontsize=12, fontweight='bold', alpha=0.9,
                    bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7))
    else:
        plt.annotate(word, (vectors_2d[i, 0], vectors_2d[i, 1]), 
                    fontsize=9, alpha=0.6)

plt.title("Word2Vec Embeddings (2D Projektion)", fontsize=14)
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

**Beobachtung:**
- Ähnliche Wörter (Hund, Katze) liegen näher beieinander
- Verschiedene Kategorien (Tiere vs. Fahrzeuge) sind getrennt

## 5. Die fundamentale Limitation: Ein Wort = Ein Vektor

**Problem:** Word2Vec gibt jedem Wort EINEN festen Vektor, egal in welchem Kontext.

In [ ]:
# Beispiel: "Bank" hat IMMER denselben Vektor
# Auch wenn wir zusätzliche Sätze mit unterschiedlichen Bedeutungen hätten:

bank_sentences = [
    ["ich", "gehe", "zur", "bank", "und", "hole", "geld"],  # Geldinstitut
    ["ich", "sitze", "auf", "der", "bank", "im", "park"],   # Sitzmöbel
]

# Neues Modell mit Bank-Beispielen
extended_sentences = sentences + bank_sentences * 10  # Auch Bank-Sätze wiederholen
model_bank = Word2Vec(extended_sentences, vector_size=150, window=7, min_count=1, sg=1, epochs=500)

# "bank" hat nur EINEN Vektor
if "bank" in model_bank.wv:
    print("Vektor für 'bank' (erste 10 Dimensionen):")
    print(model_bank.wv["bank"][:10])
    print("\nDieser Vektor ist IDENTISCH in beiden Kontexten!")
    print("Das Modell kann nicht zwischen 'Geldinstitut' und 'Sitzmöbel' unterscheiden.")
else:
    print("Wort 'bank' nicht im Vokabular (zu wenig Daten)")

## 🎯 Takeaway

**Word2Vec Vorteile:**
- ✅ Bedeutung durch Position im Vektorraum
- ✅ Vektor-Arithmetik funktioniert (König - Mann + Frau ≈ Königin)
- ✅ Ähnliche Wörter haben ähnliche Vektoren

**Word2Vec Limitation:**
- ❌ Ein Wort = Ein Vektor (kontextunabhängig)
- ❌ "Bank" hat denselben Vektor bei "Geld holen" und "im Park sitzen"
- ❌ Polysemie (mehrdeutige Wörter) kann nicht abgebildet werden

**Lösung:** Wir brauchen **kontextabhängige Embeddings** → Das führt uns zu BERT (Notebook 2)!